# Preparation II(3) - Getting Our Own Company Information

In this part, we use the list we cralwed from glassdoor website in 'preparation I' using the same method.

We are doing this to show that the methodology is applicable generally.

We are not using this dataset as it's too small compared with the other one, expect for a few palces where big datasets would cause problems including calculation speed and visualization problem.

In [1]:
import  requests
import urllib.request
import csv
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import time

In [22]:
?pd.read_csv

In [50]:
#Employer.csv contains all the employers retrieved via web scraping job posts. 
#Here will be using those values for the 'employer' parameter while making request via Glassdoor API

employer_df = pd.read_csv("own_companies.csv")
employer_df.head()

,Unnamed: 0,0
0,0,Bain & Company
1,1,Zoom Video Communications
2,2,In-N-Out Burger
3,3,Procore Technologies
4,4,Boston Consulting Group


In [51]:
employer_df.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [52]:
employer_df.drop(['Unnamed: 0'],axis=1,inplace=True)
print(employer_df)

                                         0
0                           Bain & Company
1                Zoom Video Communications
2                          In-N-Out Burger
3                     Procore Technologies
4                  Boston Consulting Group
5                                 LinkedIn
6                                 Facebook
7                                   Google
8                                lululemon
9                       Southwest Airlines
10                              Salesforce
11                      Intuitive Surgical
12   St. Jude Children's Research Hospital
13                                  Slalom
14                   Power Home Remodeling
15                                 HubSpot
16                                DocuSign
17                       Ultimate Software
18                      McKinsey & Company
19                               Paylocity
20                               Ryan, LLC
21                 Sammons Financial Group
22         

In [64]:
employer_df.rename(columns={'0':'Employer'},inplace=True)

In [65]:
employer_df.columns

Index(['Employer'], dtype='object')

### Getting ratings for the companies in the Employer.csv

We are using public Glassdoor API id (t.p=25738) and key (t.k=iRCtcWJQamE) provided via blog post: [Get Glassdoor Data from their API using Python](https://oonlab.com/get-glassdoor-data-from-their-api-using-python). This API key was gained 3 years ago when Glassdoor had free API sign up which they don’t provide any more until further notice [(link)](https://help.glassdoor.com/article/Glassdoor-API/en_US). Also, this API key has come with following restrictions on Glassdoor API Usage: [Public API Terms of Use](https://www.glassdoor.com/crs/api/glassdoor-public-api-terms.pdf). We have decided to use this public API id and key until we get our partnership inquiry permitted.

Note: We have submitted [partnership inquiry](https://help.glassdoor.com/ContactUs/en_US) several times during the fits month of Final Team Software Project, but with no respond from Glassdoor.

Glassdoor API request, used in the blog post, retrieves data about companies ratings. By reading [Glassdoor API Overview](https://www.glassdoor.com/developer/index.htm) and guidance in regards to parameters, we come to conclusion that by changing value of parameter ‘action’ from ‘action = employer’ to:
‘action = salaries’ we will fetch data about salaries, and
‘action = interviews’ we will fetch data about interview difficulty.

#### Job Postings Data
We weren’t able to find a value of parameter ‘action’ that would retrieve data about job postings, hence we decided to use web scraping for that quest. Tech topics from Meetup were used to retrieve corresponding job posts for Dublin. 

In [66]:
URLs=[]
for i in range(employer_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'employers','employer':employer_df['Employer'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    print(r.url)
    URLs.append(r.url)
    
len(URLs)

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Bain+%26+Company
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Zoom+Video+Communications
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=In-N-Out+Burger
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Procore+Technologies
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Boston+Consulting+Group
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=LinkedIn
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Facebook
http://api.glassdoor.com/api/api.htm?t

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Paycom
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=John+Deere
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=NewYork-Presbyterian+Hospital
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Oshkosh+Corporation
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Memorial+Sloan+Kettering
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=VIPKid
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=MD+Anderson+Cancer+Center
http://api.glassdoor.com/api/api.h

99

In [67]:
URLs[47]

'http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=SpaceX'

In [68]:
file = open('URLs_employers_own','w');
file.write(str(URLs));
file.close();

In [69]:
employer_df[16:17]

,Employer
16,DocuSign


In [71]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ratings_list_nested = []
for i in range(len(URLs)):
    print(i)
    url = URLs[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response, 'lxml')
    ratings_data=json.loads(soup.p.get_text())
    ratings_list_nested.append(pd.DataFrame(ratings_data['response']['employers']))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [72]:
hdr = {'User-Agent': 'Mozilla/5.0'}
for i in range(len(URLs)-1274):
    print(1274+i)
    url = URLs[1274+i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep(0.3)
    soup = BeautifulSoup(response, 'lxml')
    ratings_data=json.loads(soup.p.get_text())
    ratings_list_nested.append(pd.DataFrame(ratings_data['response']['employers']))

In [73]:
ratings_list_nested[10]

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,id,industry,industryId,industryName,...,overallRating,parentEmployer,ratingDescription,recommendToFriendRating,sectorId,sectorName,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,4.2,"{'name': 'Marc Benioff and Keith Block', 'titl...",4.4,4.5,True,{'attributionURL': 'https://www.glassdoor.com/...,11159,Computer Hardware & Software,200060.0,Computer Hardware & Software,...,4.3,NaN,Very Satisfied,86,10013.0,Information Technology,4.1,https://media.glassdoor.com/sqll/11159/salesfo...,www.salesforce.com,4.1
1,3.7,"{'name': 'Matthew Cook', 'title': 'CEO', 'numb...",4.1,4.6,False,{'attributionURL': 'https://www.glassdoor.com/...,424039,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,3.9,NaN,Satisfied,75,10006.0,Business Services,3.6,https://media.glassdoor.com/sqll/424039/the-sa...,www.thesalesforcegroup.com,3.9
2,3.8,NaN,3.0,3.7,False,{'attributionURL': 'https://www.glassdoor.com/...,1085275,,NaN,NaN,...,4.1,NaN,Very Satisfied,73,NaN,NaN,3.5,https://media.glassdoor.com/sqll/1085275/ct-co...,www.ctconsult.ru,3.5
3,4.0,NaN,2.0,5.0,False,{'attributionURL': 'https://www.glassdoor.com/...,1736863,,NaN,NaN,...,4.0,NaN,Satisfied,100,NaN,NaN,2.0,https://media.glassdoor.com/sqll/1736863/globa...,www.gtsrecruit.com,2.0
4,0.0,NaN,0.0,0.0,False,NaN,2469416,,NaN,NaN,...,0,NaN,Not Applicable,0,NaN,NaN,0.0,,,0.0
5,3.9,"{'name': 'Goolshun Belut', 'title': 'Founder',...",4.3,4.3,False,NaN,898050,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,3.6,NaN,Satisfied,100,10006.0,Business Services,3.4,https://media.glassdoor.com/sqll/898050/jobs-i...,www.jobsinsalesforce.com,3.4
6,0.0,NaN,0.0,0.0,False,NaN,3067985,,NaN,NaN,...,0,NaN,Not Applicable,0,NaN,NaN,0.0,,www.salesforceadvisers.com,0.0
7,3.6,"{'name': 'Debashis Chatterjee', 'title': 'CEO'...",3.2,3.9,False,{'attributionURL': 'https://www.glassdoor.com/...,136340,IT Services,200064.0,IT Services,...,3.8,NaN,Satisfied,78,10013.0,Information Technology,3.3,https://media.glassdoor.com/sqll/136340/mindtr...,www.mindtree.com,3.7
8,3.7,"{'name': 'Matt Meents', 'title': 'CEO', 'numbe...",3.5,3.9,False,{'attributionURL': 'https://www.glassdoor.com/...,756600,Consulting,200028.0,Consulting,...,3.8,"{'id': 136340, 'name': 'Mindtree', 'relationsh...",Satisfied,66,10006.0,Business Services,3.7,https://media.glassdoor.com/sqll/756600/magnet...,www.magnet360.com,3.8
9,2.7,"{'name': 'Rebecca Lowde', 'title': 'Chief Exec...",2.6,3.1,False,{'attributionURL': 'https://www.glassdoor.com/...,309536,Advertising & Marketing,200022.0,Advertising & Marketing,...,3.3,NaN,OK,59,10006.0,Business Services,2.9,https://media.glassdoor.com/sqll/309536/salmat...,www.salmat.com.au,3.4


In [74]:
len(ratings_list_nested)

99

In [75]:
company_rating_df_nested = pd.DataFrame()
for i in range(len(ratings_list_nested)):
    company_rating_df_nested=company_rating_df_nested.append(ratings_list_nested[i])
company_rating_df_nested

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,id,industry,industryId,industryName,...,overallRating,parentEmployer,ratingDescription,recommendToFriendRating,sectorId,sectorName,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,4.7,"{'name': 'Manny Maceda', 'title': 'Worldwide M...",4.6,4.7,True,{'attributionURL': 'https://www.glassdoor.com/...,3752,Consulting,200028.0,Consulting,...,4.6,NaN,Very Satisfied,94,10006.0,Business Services,4.5,https://media.glassdoor.com/sqll/3752/bain-and...,www.bain.com,3.7
1,4.3,"{'name': 'Glenn Cotton', 'title': 'President',...",4.2,4.4,False,{'attributionURL': 'https://www.glassdoor.com/...,147915,Other Retail Stores,200115.0,Other Retail Stores,...,4.4,NaN,Very Satisfied,89,10022.0,Retail,4.3,https://media.glassdoor.com/sqll/147915/bath-f...,www.bathfitter.com,4.0
2,4.0,{'name': 'Jonathan Lavine and John Connaughton...,4.4,4.0,False,{'attributionURL': 'https://www.glassdoor.com/...,3930,Investment Banking & Asset Management,200053.0,Investment Banking & Asset Management,...,4.1,NaN,Very Satisfied,85,10010.0,Finance,4.2,https://media.glassdoor.com/sqll/3930/bain-cap...,www.baincapital.com,3.5
3,3.6,"{'name': 'Bill Riss', 'title': 'CEO & Owner', ...",3.0,3.8,False,{'attributionURL': 'https://www.glassdoor.com/...,832466,Real Estate,200094.0,Real Estate,...,3.8,NaN,Satisfied,74,10020.0,Real Estate,3.8,https://media.glassdoor.com/sqll/832466/coldwe...,www.coldwellbankerbain.com,3.1
4,3.5,"{'name': 'Alejandro Bernabé', 'title': 'CEO', ...",3.2,3.3,False,{'attributionURL': 'https://www.glassdoor.com/...,152773,"Hotels, Motels, & Resorts",200140.0,"Hotels, Motels, & Resorts",...,3.6,NaN,Satisfied,65,10025.0,Travel & Tourism,3.0,https://media.glassdoor.com/sqll/152773/kempin...,www.kempinski.com,3.1
5,4.3,"{'name': 'David Heron', 'title': 'CEO', 'numbe...",4.1,4.3,False,{'attributionURL': 'https://www.glassdoor.com/...,758654,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,4.3,NaN,Very Satisfied,82,10006.0,Business Services,4.3,https://media.glassdoor.com/sqll/758654/wilton...,www.wilton-bain.com,4.0
6,1.4,"{'name': 'Mark Bain', 'title': 'President', 'n...",1.4,1.5,False,{'attributionURL': 'https://www.glassdoor.com/...,845241,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,2.0,NaN,Dissatisfied,19,10006.0,Business Services,1.7,,www.bainstaffing.com,1.2
7,3.4,NaN,3.4,4.6,False,{'attributionURL': 'https://www.glassdoor.com/...,977970,,NaN,NaN,...,4.1,NaN,Very Satisfied,100,NaN,NaN,2.9,https://media.glassdoor.com/sqll/977970/lbj-fo...,www.lbjlibrary.org,4.6
8,2.9,NaN,2.6,4.2,False,{'attributionURL': 'https://www.glassdoor.com/...,1293213,,NaN,NaN,...,3.8,NaN,Satisfied,39,NaN,NaN,3.2,,www.baindentalgroup.com,2.6
9,3.0,NaN,3.0,4.0,False,{'attributionURL': 'https://www.glassdoor.com/...,1068223,,NaN,NaN,...,2.2,NaN,Dissatisfied,0,NaN,NaN,2.0,,www.baindermatology.com,3.0


In [76]:
company_rating_df_nested["name"]

0                                Bain & Company
1                                   Bath Fitter
2                                  Bain Capital
3                          Coldwell Banker Bain
4                                     Kempinski
5                                 Wilton & Bain
6                        Bain Staffing Services
7                                LBJ Foundation
8                             Bain Dental Group
9                         Bain Dermatology, P.A
0                     Zoom Video Communications
0                               In-N-Out Burger
0                          Procore Technologies
0                       Boston Consulting Group
1               Greater Boston Consulting Group
2                  Hamilton & Boston Consulting
3                          BCG Digital Ventures
4                                  Alira Health
5                  Boston Industrial Consulting
6    Boston Management Consulting International
7        Tehran â€“ Boston Consulting En

In [77]:
company_rating_df_nested.to_csv(r'own_companies_rating.csv')

In [78]:
#Adding new columns because of the nested featuredReview column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list=['review_URL','review_ID','review_currentJob','review_DateTime',
              'review_jobTitle','review_location','review_pros','review_cons','review_overall']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list])

In [79]:
split_featuredReview = company_rating_df_nested['featuredReview'].apply(pd.Series) # split the featuredReview column from company_rating_df_nested
split_featuredReview = split_featuredReview.reset_index(drop=True) # reset index

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['review_URL'].iloc[i]=split_featuredReview['attributionURL'][i]
    company_rating_df_nested['review_ID'].iloc[i]=split_featuredReview['id'][i]
    company_rating_df_nested['review_currentJob'].iloc[i]=split_featuredReview['currentJob'][i]
    company_rating_df_nested['review_DateTime'].iloc[i]=split_featuredReview['reviewDateTime'][i]
    company_rating_df_nested['review_jobTitle'].iloc[i]=split_featuredReview['jobTitle'][i]
    company_rating_df_nested['review_location'].iloc[i]=split_featuredReview['location'][i]
    company_rating_df_nested['review_pros'].iloc[i]=split_featuredReview['pros'][i]
    company_rating_df_nested['review_cons'].iloc[i]=split_featuredReview['cons'][i]
    company_rating_df_nested['review_overall'].iloc[i]=split_featuredReview['overall'][i]


//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [80]:
#Adding new columns because of the nested ceo column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list2=['ceo_name','ceo_title','ceo_numberOfRatings','ceo_approval_rate','ceo_Photo_URL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list2])

In [81]:
split_ceo = company_rating_df_nested['ceo'].apply(pd.Series) # split the ceo column from company_rating_df_nested
split_ceo = split_ceo.reset_index(drop=True) # reset index
split_ceo_2 = split_ceo['image'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['ceo_name'].iloc[i]=split_ceo['name'][i]
    company_rating_df_nested['ceo_title'].iloc[i]=split_ceo['title'][i]
    company_rating_df_nested['ceo_numberOfRatings'].iloc[i]=split_ceo['numberOfRatings'][i]
    company_rating_df_nested['ceo_approval_rate'].iloc[i]=split_ceo['pctApprove'][i]
    company_rating_df_nested['ceo_Photo_URL'].iloc[i]=split_ceo_2['src'][i]
#company_rating_df_nested.head()

In [82]:
#Adding new columns because of the nested parentEmployer column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list3=['parentEmployer_name','parentEmployer_ID','parentEmployer_logoURL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list3])

In [83]:
split_parentEmployer = company_rating_df_nested['parentEmployer'].apply(pd.Series) # split the parentEmployer column from company_rating_df_nested
split_parentEmployer = split_parentEmployer.reset_index(drop=True) # reset index
split_parentEmployer_2 = split_parentEmployer['logo'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['parentEmployer_name'].iloc[i]=split_parentEmployer['name'][i]
    company_rating_df_nested['parentEmployer_ID'].iloc[i]=split_parentEmployer['id'][i]
    company_rating_df_nested['parentEmployer_logoURL'].iloc[i]=split_parentEmployer_2['normalUrl'][i]


#company_rating_df_nested.head()

In [84]:
company_rating_df_nested

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,id,industry,industryId,industryName,...,review_cons,review_overall,ceo_name,ceo_title,ceo_numberOfRatings,ceo_approval_rate,ceo_Photo_URL,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL
0,4.7,"{'name': 'Manny Maceda', 'title': 'Worldwide M...",4.6,4.7,True,{'attributionURL': 'https://www.glassdoor.com/...,3752,Consulting,200028.0,Consulting,...,Here's how you know you've made it at Bain. T...,5.0,Manny Maceda,Worldwide Managing Partner,580.0,99.0,https://media.glassdoor.com/people/sqll/3752/b...,NaN,NaN,NaN
1,4.3,"{'name': 'Glenn Cotton', 'title': 'President',...",4.2,4.4,False,{'attributionURL': 'https://www.glassdoor.com/...,147915,Other Retail Stores,200115.0,Other Retail Stores,...,The occasional bad seed but there is always on...,5.0,Glenn Cotton,President,109.0,94.0,https://media.glassdoor.com/people/sqll/147915...,NaN,NaN,NaN
2,4.0,{'name': 'Jonathan Lavine and John Connaughton...,4.4,4.0,False,{'attributionURL': 'https://www.glassdoor.com/...,3930,Investment Banking & Asset Management,200053.0,Investment Banking & Asset Management,...,There are some people who are not pulling thei...,5.0,Jonathan Lavine and John Connaughton,Co-Managing Partners,25.0,99.0,NaN,NaN,NaN,NaN
3,3.6,"{'name': 'Bill Riss', 'title': 'CEO & Owner', ...",3.0,3.8,False,{'attributionURL': 'https://www.glassdoor.com/...,832466,Real Estate,200094.0,Real Estate,...,I felt pressured to do well.,5.0,Bill Riss,CEO & Owner,22.0,81.0,NaN,NaN,NaN,NaN
4,3.5,"{'name': 'Alejandro Bernabé', 'title': 'CEO', ...",3.2,3.3,False,{'attributionURL': 'https://www.glassdoor.com/...,152773,"Hotels, Motels, & Resorts",200140.0,"Hotels, Motels, & Resorts",...,Difficult to change between departments,4.0,Alejandro Bernabé,CEO,45.0,89.0,https://media.glassdoor.com/people/sqll/152773...,NaN,NaN,NaN
5,4.3,"{'name': 'David Heron', 'title': 'CEO', 'numbe...",4.1,4.3,False,{'attributionURL': 'https://www.glassdoor.com/...,758654,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,Can be long hours but the time tends to fly by...,5.0,David Heron,CEO,7.0,100.0,https://media.glassdoor.com/people/sqll/758654...,NaN,NaN,NaN
6,1.4,"{'name': 'Mark Bain', 'title': 'President', 'n...",1.4,1.5,False,{'attributionURL': 'https://www.glassdoor.com/...,845241,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,someone esle say Bain is the wi\ost places to ...,5.0,Mark Bain,President,2.0,45.0,NaN,NaN,NaN,NaN
7,3.4,NaN,3.4,4.6,False,{'attributionURL': 'https://www.glassdoor.com/...,977970,,NaN,NaN,...,"I can't think of any, I love working here!",5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.9,NaN,2.6,4.2,False,{'attributionURL': 'https://www.glassdoor.com/...,1293213,,NaN,NaN,...,None that I can think of.,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,NaN,3.0,4.0,False,{'attributionURL': 'https://www.glassdoor.com/...,1068223,,NaN,NaN,...,unable to fill schedule and no time off. Dr ex...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
#dropping columns'ceo','featuredReview', 'parentEmployer'
company_rating_df2=company_rating_df_nested.drop(['ceo','featuredReview', 'parentEmployer'], axis=1)

In [92]:
#Removing duplicate rows
company_rating_df = company_rating_df2.drop_duplicates(subset=None, keep='first', inplace=False)

In [93]:
company_rating_df.to_csv('rating_own.csv', sep = ',', index = False) # create csv for salaries
company_rating_df

,careerOpportunitiesRating,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,id,industry,industryId,industryName,isEEP,name,...,review_cons,review_overall,ceo_name,ceo_title,ceo_numberOfRatings,ceo_approval_rate,ceo_Photo_URL,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL
0,4.7,4.6,4.7,True,3752,Consulting,200028.0,Consulting,True,Bain & Company,...,Here's how you know you've made it at Bain. T...,5.0,Manny Maceda,Worldwide Managing Partner,580.0,99.0,https://media.glassdoor.com/people/sqll/3752/b...,NaN,NaN,NaN
1,4.3,4.2,4.4,False,147915,Other Retail Stores,200115.0,Other Retail Stores,True,Bath Fitter,...,The occasional bad seed but there is always on...,5.0,Glenn Cotton,President,109.0,94.0,https://media.glassdoor.com/people/sqll/147915...,NaN,NaN,NaN
2,4.0,4.4,4.0,False,3930,Investment Banking & Asset Management,200053.0,Investment Banking & Asset Management,False,Bain Capital,...,There are some people who are not pulling thei...,5.0,Jonathan Lavine and John Connaughton,Co-Managing Partners,25.0,99.0,NaN,NaN,NaN,NaN
3,3.6,3.0,3.8,False,832466,Real Estate,200094.0,Real Estate,False,Coldwell Banker Bain,...,I felt pressured to do well.,5.0,Bill Riss,CEO & Owner,22.0,81.0,NaN,NaN,NaN,NaN
4,3.5,3.2,3.3,False,152773,"Hotels, Motels, & Resorts",200140.0,"Hotels, Motels, & Resorts",False,Kempinski,...,Difficult to change between departments,4.0,Alejandro Bernabé,CEO,45.0,89.0,https://media.glassdoor.com/people/sqll/152773...,NaN,NaN,NaN
5,4.3,4.1,4.3,False,758654,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,False,Wilton & Bain,...,Can be long hours but the time tends to fly by...,5.0,David Heron,CEO,7.0,100.0,https://media.glassdoor.com/people/sqll/758654...,NaN,NaN,NaN
6,1.4,1.4,1.5,False,845241,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,False,Bain Staffing Services,...,someone esle say Bain is the wi\ost places to ...,5.0,Mark Bain,President,2.0,45.0,NaN,NaN,NaN,NaN
7,3.4,3.4,4.6,False,977970,,NaN,NaN,False,LBJ Foundation,...,"I can't think of any, I love working here!",5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.9,2.6,4.2,False,1293213,,NaN,NaN,False,Bain Dental Group,...,None that I can think of.,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,3.0,4.0,False,1068223,,NaN,NaN,False,"Bain Dermatology, P.A",...,unable to fill schedule and no time off. Dr ex...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
